### Initialising Libraries

In [1]:
#initialising libraries
import json
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import glob
import os
import re

pd.set_option("display.max_colwidth", 70000)

#### setting folder for analysis and concat all datewise tweet files.
input dialog box asks for path, just go to the parent folder where `json` files are located and select that folder location. 
the rest of the things will be taken care of automatically.


In [2]:
nof = int(input("Enter 1 for single file or 2 for multiple files: "))

if nof == 1:
    file_path = input('Enter the file path: ')
    tweets = pd.read_csv(file_path, lineterminator='\n')
elif nof == 2:
    folder_path2 = input("Enter the folder path: ")
    folder_path = folder_path2 + "/" + "datewise"
    all_files = glob.iglob(os.path.join(folder_path, "*.csv"))
    tweets = pd.concat((pd.read_csv(f, lineterminator='\n', sep = ';') for f in all_files))
else:
    print("Path not found.")

Enter 1 for single file or 2 for multiple files:  2
Enter the folder path:  /media/ambijat/Datrat/ipytoneeee/CNUS


#### Inspecting the datatype.

In [3]:
tweets.dtypes

user                    object
reuser                  object
full_text               object
range                    int64
created_at              object
id                       int64
in_reply_to_user_id    float64
user.location           object
date                    object
date2                    int64
dtype: object

#### total number of tweets.

In [4]:
print(len(tweets))

116017


#### choosing `full_text` selection.

In [5]:
wrench= tweets['full_text'].copy()
t1 = "full_text"

### Processing tweets for various quantitative analysis.
##### sending to a text file for word count analysis.

In [6]:
wrench.to_csv(r'wrench.txt', header=False, index=None, sep=' ', mode='a')

#### Using `bash` script for extracting the frequency of words in overall collection of tweets.

In [7]:
!rm *.tmp

!cat wrench.txt | sed 's/[[:space:]]/\n/g' | sed 's/^[ \t]*//' | sed 's/[ \t]*$//' | \
sed '/^$/d'| tr -cd '[:alnum:] [:space:]' >> rvd2.tmp

!awk 'length > 2' < rvd2.tmp | awk '{print tolower($0)}' > rvd22.tmp

!grep -vwF -f exclude.exd < rvd22.tmp > rvd11.tmp


!cat rvd11.tmp | sort | uniq -c | sort -nr | head -n 250 > lsfdout.txt

!rm *.tmp

rm: cannot remove '*.tmp': No such file or directory
sort: write failed: 'standard output': Broken pipe
sort: write error


#### this gives the frequency of words used in the tweets. The treemap can be developed.
##### maximum number of times words occuring and listing them as rating.

In [8]:
rating = pd.read_csv('lsfdout.txt', header=None, names=["words"])
rating.head(30)

,words
0,92210 hong
1,80664 kong
2,57871 china
3,15115 law
4,13399 security
5,12065 chinas
6,10767 johnson
7,8715 people
8,8421 national
9,8156 que


##### Using `str.lstrp` instead of `str.replace` for removing intial whitespace and then splitting the columns.
##### take note of `str.split` where `n=1` represents splitting at the first instance of space and not on later ones.

In [9]:
r2 = rating.copy()
r2.words = r2.words.str.lstrip()
r2.dropna(inplace = True) 
new = r2['words'].str.split(" ", n = 1, expand = True) 
r2['count']= new[0] 
r2['wordlist']= new[1] 
r2.drop(columns =['words'], inplace = True) 
r2.head(20)

,count,wordlist
0,92210,hong
1,80664,kong
2,57871,china
3,15115,law
4,13399,security
5,12065,chinas
6,10767,johnson
7,8715,people
8,8421,national
9,8156,que


### exporting the most words to csv file.

In [10]:
r2b = r2.head(50).copy()
mostword = os.path.join(folder_path2 + "/" + "processed2", 'mostword.csv')
r2b.to_csv(mostword, index = False, header = True, encoding = 'utf-8', sep = ';')

##### maximum number of location used for tweets.

In [11]:
print(tweets['user.location'].value_counts().nlargest(20))

Hong Kong                  11905
香港                          1464
United States                998
London, England              503
London                       460
United Kingdom               395
India                        375
USA                          359
Internet                     319
Canada                       302
England, United Kingdom      277
California, USA              273
Washington, DC               246
New York, NY                 221
New York, USA                221
Argentina                    188
Los Angeles, CA              186
Venezuela                    186
Toronto, Ontario             183
HK                           183
Name: user.location, dtype: int64


In [12]:
#exporting the most userloc to csv file.
userloc = pd.DataFrame(tweets['user.location'].value_counts().nlargest(30))
userloc.reset_index(level=0, inplace=True)
mostloc = os.path.join(folder_path2 + "/" + "processed2", 'mostloc.csv')
userloc.to_csv(mostloc, index = False, header = True, encoding = 'utf-8', sep = ';')

#### Making use of frequency of words to grasp relevant tweets by boolean exercise.

In [13]:
sof = 0
sof = int(input("For Custom words Press 1 or 0 to Continue: "))

if sof == 1:
    k1 = input("Enter first word: ")
    k2 = input("Enter second word: ")
    k3 = input("Enter third word: ")
elif sof == 0:
    k1 = r2.wordlist[0]
    k2 = r2.wordlist[1]
    k3 = r2.wordlist[2]
else:
    print("No choice given.")

For Custom words Press 1 or 0 to Continue:  1
Enter first word:  hong
Enter second word:  china
Enter third word:  law


#### `k1`, `k2`, & `k3` are the 3 maximun frequency words in the tweet text.
##### refer to `cell no. 5` for `t1` variable. `case=False` stands for ignorecase.

In [14]:
tweets[k1] = tweets[t1].str.contains(k1, case=False)
tweets[k2] = tweets[t1].str.contains(k2, case=False)
tweets[k3] = tweets[t1].str.contains(k3, case=False)

#### The above method can also be done by defining a function. 
##### Same example is repeated. `t1` refers to `full_text`.

In [15]:
def wordcheck(series, key):
    return series.str.contains(key, case=False)

for key in (k1,k2,k3):
    tweets[key +'2'] = wordcheck(tweets[t1], key)

##### Inspecting the output.

In [16]:
tweets.head(20)

,user,reuser,full_text,range,created_at,id,in_reply_to_user_id,user.location,date,date2,hong,china,law,hong2,china2,law2
0,namjoonayo,cannacae,RT @cannacae: i know BLM is important rn but i URGE you to look at Hong Kong for even a second. America is taking away its preferential tre…,140,Mon Jun 01 23:59:57 +0000 2020,1267606846824542200,NaN,NaN,2020-06-01 23:59:57+00:00,1062020,True,False,False,True,False,False
1,ngdasokau,BarrySheerman,RT @BarrySheerman: Will the Johnson Government now wake up to the need for a coordinated global response to the malevolence of China’s acti…,140,Mon Jun 01 23:59:57 +0000 2020,1267606845763367000,NaN,Hong Kong,2020-06-01 23:59:57+00:00,1062020,False,True,False,False,True,False
2,geoffreypcdog,benedictrogers,RT @benedictrogers: Seven former foreign secretaries urge Boris Johnson to lead global response to 'deteriorating' Hong Kong situation http…,140,Mon Jun 01 23:59:57 +0000 2020,1267606843322331100,NaN,NaN,2020-06-01 23:59:57+00:00,1062020,True,False,False,True,False,False
3,Marisa15061974,WDVL4,RT @WDVL4: China pidió a empresas estatales que suspendan compras de soja y carne de cerdo de EEUU luego de que Washington anunciase que el…,140,Mon Jun 01 23:59:56 +0000 2020,1267606839547625500,NaN,NaN,2020-06-01 23:59:56+00:00,1062020,False,True,False,False,True,False
4,Tony13945454,nytimesworld,"RT @nytimesworld: The gathering to remember Tiananmen, held annually since 1990, had become a major rallying point for Hong Kongers worried…",140,Mon Jun 01 23:59:55 +0000 2020,1267606835428593700,NaN,Hong Kong,2020-06-01 23:59:55+00:00,1062020,True,False,False,True,False,False
5,kangaroolooloo,SenatorLankford,RT @SenatorLankford: while Americans protest injustice-China bans in Hong Kong any remembrance of the 1989 massacre in Tiananmen Square- fr…,140,Mon Jun 01 23:59:50 +0000 2020,1267606815530823700,NaN,Hong Kong,2020-06-01 23:59:50+00:00,1062020,True,True,False,True,True,False
6,Alankawing,bopinion,"RT @bopinion: Taiwan, the U.S., Canada and Australia — democracies that have spoken up in support of the rights of Hong Kongers — need to b…",140,Mon Jun 01 23:59:48 +0000 2020,1267606806215356400,NaN,NaN,2020-06-01 23:59:48+00:00,1062020,True,False,False,True,False,False
7,Tony13945454,MailOnline,RT @MailOnline: Hong Kong's pro-Beijing government is criticised for 'stifling its citizens' freedom of expression' https://t.co/Gro13tb3A1,139,Mon Jun 01 23:59:48 +0000 2020,1267606805720400000,NaN,Hong Kong,2020-06-01 23:59:48+00:00,1062020,True,False,False,True,False,False
8,Chowcch1,bopinion,"RT @bopinion: Taiwan, the U.S., Canada and Australia — democracies that have spoken up in support of the rights of Hong Kongers — need to b…",140,Mon Jun 01 23:59:48 +0000 2020,1267606805284180000,NaN,NaN,2020-06-01 23:59:48+00:00,1062020,True,False,False,True,False,False
9,FidelisToJusti1,FidelisToJusti1,RT @FidelisToJusti1: What @realDonaldTrump is threatening if governors don't do what he wants is to emulate Communist China's crackdown on…,139,Mon Jun 01 23:59:45 +0000 2020,1267606795637461000,NaN,"Queens, NY",2020-06-01 23:59:45+00:00,1062020,False,True,False,False,True,False


##### Finding the number of tweets for which the `k1` term occurs, `k2` occurs and `k3` occurs.

In [17]:
print(tweets[k1].value_counts()[True])
print(tweets[k2].value_counts()[True])
print(tweets[k3].value_counts()[True])
#
print(tweets[k1 +'2'].value_counts()[True])
print(tweets[k2 +'2'].value_counts()[True])
print(tweets[k3 +'2'].value_counts()[True])

91180
68439
17002
91180
68439
17002


#### Now finding the tweets which are combination of one or more than one keywords.

In [18]:
tweets[k2+k1] = tweets[t1].str.contains(k2 or k1, case=False)
tweets[k3+k1] = tweets[t1].str.contains(k3 or k1, case=False)

tweets[k2+k1+'2'] = tweets[t1].str.contains(k2 or k1, case=False)
tweets[k3+k1+'2'] = tweets[t1].str.contains(k3 or k1, case=False)

#### If you notice these scores then `k2+k1` and `k3+k1` are the ones that have similar scores for their combination, just as their individual `k2` and `k3` scores.

In [19]:
print(tweets[k2+k1].value_counts()[True])
print(tweets[k3+k1].value_counts()[True])
#
print(tweets[k2+k1+'2'].value_counts()[True])
print(tweets[k3+k1+'2'].value_counts()[True])

68439
17002
68439
17002


#### The following score is basically `and` boolean operator in between `k2+k1` and `k3+k1` and `k1` values.

In [20]:
print(tweets[tweets[k2+k1]][k1].value_counts()[True])
print(tweets[tweets[k3+k1]][k1].value_counts()[True])
#
print(tweets[tweets[k2+k1+'2']][k1].value_counts()[True])
print(tweets[tweets[k3+k1+'2']][k1].value_counts()[True])

57681
14506
57681
14506


### Exporting the tweets that contain most of the three keywords.

In [21]:
separ= tweets[(tweets[k2+k1+'2'] == True) | (tweets[k3+k1+'2'] == True)].copy()

In [22]:
separ.head()

,user,reuser,full_text,range,created_at,id,in_reply_to_user_id,user.location,date,date2,hong,china,law,hong2,china2,law2,chinahong,lawhong,chinahong2,lawhong2
1,ngdasokau,BarrySheerman,RT @BarrySheerman: Will the Johnson Government now wake up to the need for a coordinated global response to the malevolence of China’s acti…,140,Mon Jun 01 23:59:57 +0000 2020,1267606845763367000,NaN,Hong Kong,2020-06-01 23:59:57+00:00,1062020,False,True,False,False,True,False,True,False,True,False
3,Marisa15061974,WDVL4,RT @WDVL4: China pidió a empresas estatales que suspendan compras de soja y carne de cerdo de EEUU luego de que Washington anunciase que el…,140,Mon Jun 01 23:59:56 +0000 2020,1267606839547625500,NaN,NaN,2020-06-01 23:59:56+00:00,1062020,False,True,False,False,True,False,True,False,True,False
5,kangaroolooloo,SenatorLankford,RT @SenatorLankford: while Americans protest injustice-China bans in Hong Kong any remembrance of the 1989 massacre in Tiananmen Square- fr…,140,Mon Jun 01 23:59:50 +0000 2020,1267606815530823700,NaN,Hong Kong,2020-06-01 23:59:50+00:00,1062020,True,True,False,True,True,False,True,False,True,False
9,FidelisToJusti1,FidelisToJusti1,RT @FidelisToJusti1: What @realDonaldTrump is threatening if governors don't do what he wants is to emulate Communist China's crackdown on…,139,Mon Jun 01 23:59:45 +0000 2020,1267606795637461000,NaN,"Queens, NY",2020-06-01 23:59:45+00:00,1062020,False,True,False,False,True,False,True,False,True,False
10,kangaroolooloo,next_china,RT @next_china: Hong Kong’s finance workers are seeking to organize a strike to protest against China’s plans to implement new security law…,140,Mon Jun 01 23:59:45 +0000 2020,1267606793041047600,NaN,Hong Kong,2020-06-01 23:59:45+00:00,1062020,True,True,True,True,True,True,True,True,True,True


In [23]:
print(len(separ))

73387


In [24]:
print(separ['user.location'].value_counts().nlargest(50))

Hong Kong                         7445
香港                                 930
United States                      617
London, England                    355
London                             329
United Kingdom                     266
India                              255
Internet                           224
USA                                222
Canada                             196
England, United Kingdom            186
California, USA                    168
Washington, DC                     161
Argentina                          142
New York, NY                       140
New York, USA                      137
Buenos Aires, Argentina            132
Venezuela                          122
Australia                          118
HK                                 116
UK                                 115
Toronto, Ontario                   113
Los Angeles, CA                    105
New Delhi, India                   101
Worldwide                           98
Singapore                

### Separating and counting the `Retweets`.
##### this regex expression `^(?=.*\bRT\b\s@\b).*$` is good only for direct expression and not for reference variable.

In [25]:
separ['RT'] = separ[t1].str.contains(r"^(?:RT\s@\w+:)", case=False) == True

In [26]:
print(separ['RT'].value_counts()[False])

13523


In [27]:
print(separ['RT'].value_counts()[True])

59864


#### Extracting the `urls` from the text.

In [28]:
separ['URLS'] = separ[t1].str.extract(r'(https?://[^\s<>"]+|www\.[^\s<>"]+)', expand = False)

In [29]:
separ.head()

,user,reuser,full_text,range,created_at,id,in_reply_to_user_id,user.location,date,date2,...,law,hong2,china2,law2,chinahong,lawhong,chinahong2,lawhong2,RT,URLS
1,ngdasokau,BarrySheerman,RT @BarrySheerman: Will the Johnson Government now wake up to the need for a coordinated global response to the malevolence of China’s acti…,140,Mon Jun 01 23:59:57 +0000 2020,1267606845763367000,NaN,Hong Kong,2020-06-01 23:59:57+00:00,1062020,...,False,False,True,False,True,False,True,False,True,NaN
3,Marisa15061974,WDVL4,RT @WDVL4: China pidió a empresas estatales que suspendan compras de soja y carne de cerdo de EEUU luego de que Washington anunciase que el…,140,Mon Jun 01 23:59:56 +0000 2020,1267606839547625500,NaN,NaN,2020-06-01 23:59:56+00:00,1062020,...,False,False,True,False,True,False,True,False,True,NaN
5,kangaroolooloo,SenatorLankford,RT @SenatorLankford: while Americans protest injustice-China bans in Hong Kong any remembrance of the 1989 massacre in Tiananmen Square- fr…,140,Mon Jun 01 23:59:50 +0000 2020,1267606815530823700,NaN,Hong Kong,2020-06-01 23:59:50+00:00,1062020,...,False,True,True,False,True,False,True,False,True,NaN
9,FidelisToJusti1,FidelisToJusti1,RT @FidelisToJusti1: What @realDonaldTrump is threatening if governors don't do what he wants is to emulate Communist China's crackdown on…,139,Mon Jun 01 23:59:45 +0000 2020,1267606795637461000,NaN,"Queens, NY",2020-06-01 23:59:45+00:00,1062020,...,False,False,True,False,True,False,True,False,True,NaN
10,kangaroolooloo,next_china,RT @next_china: Hong Kong’s finance workers are seeking to organize a strike to protest against China’s plans to implement new security law…,140,Mon Jun 01 23:59:45 +0000 2020,1267606793041047600,NaN,Hong Kong,2020-06-01 23:59:45+00:00,1062020,...,True,True,True,True,True,True,True,True,True,NaN


In [30]:
type(separ)
separ.dtypes

user                    object
reuser                  object
full_text               object
range                    int64
created_at              object
id                       int64
in_reply_to_user_id    float64
user.location           object
date                    object
date2                    int64
hong                      bool
china                     bool
law                       bool
hong2                     bool
china2                    bool
law2                      bool
chinahong                 bool
lawhong                   bool
chinahong2                bool
lawhong2                  bool
RT                        bool
URLS                    object
dtype: object

### Finding the range of the dates.

In [31]:
#separ['date2'] = separ.date.dt.strftime('%d%m%Y')
collect = separ['date2'].unique()
print(collect)

[1062020 2062020 3062020]


## Selecting columns for further processing.

In [32]:
separ3a = separ.iloc[:, [0, 1, 2, 3, 7, 9, 13, 14, 15, -1, -2, -3, -4]]

In [33]:
separ3a.head()

,user,reuser,full_text,range,user.location,date2,hong2,china2,law2,URLS,RT,lawhong2,chinahong2
1,ngdasokau,BarrySheerman,RT @BarrySheerman: Will the Johnson Government now wake up to the need for a coordinated global response to the malevolence of China’s acti…,140,Hong Kong,1062020,False,True,False,NaN,True,False,True
3,Marisa15061974,WDVL4,RT @WDVL4: China pidió a empresas estatales que suspendan compras de soja y carne de cerdo de EEUU luego de que Washington anunciase que el…,140,NaN,1062020,False,True,False,NaN,True,False,True
5,kangaroolooloo,SenatorLankford,RT @SenatorLankford: while Americans protest injustice-China bans in Hong Kong any remembrance of the 1989 massacre in Tiananmen Square- fr…,140,Hong Kong,1062020,True,True,False,NaN,True,False,True
9,FidelisToJusti1,FidelisToJusti1,RT @FidelisToJusti1: What @realDonaldTrump is threatening if governors don't do what he wants is to emulate Communist China's crackdown on…,139,"Queens, NY",1062020,False,True,False,NaN,True,False,True
10,kangaroolooloo,next_china,RT @next_china: Hong Kong’s finance workers are seeking to organize a strike to protest against China’s plans to implement new security law…,140,Hong Kong,1062020,True,True,True,NaN,True,True,True


### Exporting tweets with most `urls` frequency.
#### filter the dataframe for tweets with urls of highest frequency.

In [34]:
mostur = pd.DataFrame(separ3a['URLS'].value_counts().nlargest(60))
mostur.reset_index(level=0, inplace=True)

In [35]:
type(mostur)

pandas.core.frame.DataFrame

In [36]:
mostur.head()

,index,URLS
0,https://t.co/A11HGtqK0v,1056
1,https://t.co/TysDBVW8H9,673
2,https://t.co/Ek680whc2t,547
3,https://t.co/6BNz7v8w5m,506
4,https://t.co/ZxeHlOhqvh,477


### `URL` with highest frequency.

In [37]:
mostur_0 = mostur['index'].iloc[0]
print(mostur_0)

https://t.co/A11HGtqK0v


### exporting the most urls to csv file.

In [38]:
mosturls = os.path.join(folder_path2 + "/" + "processed2", 'mostur.csv')
print(mosturls)
mostur.to_csv(mosturls, index = False, header = True, encoding = 'utf-8', sep = ';')

/media/ambijat/Datrat/ipytoneeee/CNUS/processed2/mostur.csv


### Using the most `URL` to identify tweets and exporting it as a dataframe.

In [39]:
separ3b = separ3a.loc[separ['full_text'].str.contains(mostur_0, flags=re.IGNORECASE, regex=True)].copy()

In [40]:
separ3b.head()

,user,reuser,full_text,range,user.location,date2,hong2,china2,law2,URLS,RT,lawhong2,chinahong2
75,legloa1s8,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,hongkong,1062020,True,True,True,https://t.co/A11HGtqK0v,True,True,True
202,LazyLionLovesHK,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,NaN,1062020,True,True,True,https://t.co/A11HGtqK0v,True,True,True
270,Joelie92228745,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,NaN,1062020,True,True,True,https://t.co/A11HGtqK0v,True,True,True
283,Lambert11679267,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,NaN,1062020,True,True,True,https://t.co/A11HGtqK0v,True,True,True
299,Hot_L_,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,NaN,1062020,True,True,True,https://t.co/A11HGtqK0v,True,True,True


### Reducing the number of columns for export.

In [41]:
separ3c = separ3b.iloc[:, [0, 1, 2, 3, 9, -1, -2, -3, -4]]

In [42]:
separ3c.head()

,user,reuser,full_text,range,URLS,chinahong2,lawhong2,RT,URLS
75,legloa1s8,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,https://t.co/A11HGtqK0v,True,True,True,https://t.co/A11HGtqK0v
202,LazyLionLovesHK,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,https://t.co/A11HGtqK0v,True,True,True,https://t.co/A11HGtqK0v
270,Joelie92228745,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,https://t.co/A11HGtqK0v,True,True,True,https://t.co/A11HGtqK0v
283,Lambert11679267,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,https://t.co/A11HGtqK0v,True,True,True,https://t.co/A11HGtqK0v
299,Hot_L_,benedictrogers,RT @benedictrogers: BBC News - Hong Kong: Boris Johnson urged to form alliance over China security law https://t.co/A11HGtqK0v,126,https://t.co/A11HGtqK0v,True,True,True,https://t.co/A11HGtqK0v


In [43]:
mosturltweet = os.path.join(folder_path2 + "/" + "processed2", 'mosturl_tweet.csv')
separ3c.to_csv(mosturltweet, index = False, header = True, encoding = 'utf-8', sep = ';')

### `Separ3a` is split into two dataframe `sp2` and `sp3` of `_original` and `_retweets` and exported to `csv` format.

In [44]:
sp2 = separ3a[separ3a['RT'] == False]

In [45]:
print(len(sp2))

13523


In [46]:
sp3= separ3a[separ3a['RT'] == True]

In [47]:
print(len(sp3))

59864


### exporting the original tweets to csv file. `str(collect[0])` and `str(collect[-1])` are conversion of `int64` to `string`.

In [48]:
k1k2_original = os.path.join(folder_path2 + "/" + "processed2", k1+k2+ str(collect[0]) + str(collect[-1]) +'_original.csv')
print(k1k2_original)
sp2.to_csv(k1k2_original, index = False, header = True, encoding = 'utf-8', sep = ';')

/media/ambijat/Datrat/ipytoneeee/CNUS/processed2/hongchina10620203062020_original.csv


### exporting the retweets to csv file.

In [49]:
k1k2_retweets = os.path.join(folder_path2 + "/" + "processed2", k1+k2+ str(collect[0]) + str(collect[-1]) +'_retweets.csv')
print(k1k2_retweets)
sp3.to_csv(k1k2_retweets, index = False, header = True, encoding = 'utf-8', sep = ';')

/media/ambijat/Datrat/ipytoneeee/CNUS/processed2/hongchina10620203062020_retweets.csv


In [50]:
!rm *.tmp
!rm *.txt

rm: cannot remove '*.tmp': No such file or directory
